# EMMSB 2018
Prática de dinâmica molecular de complexo proteína-ligante.  
Diego Enry B. Gomes - Université de Strasbourg - France | dgomes@pq.cnpq.br


## Dias 2, 3 e 4
Na práticas a seguir vamos discutir a preparação da uma estrutura de proteína para simulações moleculares e fazer uma simulação de dinâmica molecular em complexo com um ligante finalizando com a estimativa da energia livre de ligação.

No dia 02 discutiremos:
* Conteúdo do arquivo de estrutura, comentários importantes do arquivo .PDB.
* Previsão dos estados de protonação dos resíduos de aminoácidos (PROPKa).
* Preparação da estrutura/topologia da proteína para o programa de simulação (GROMACS).
* Parametrização de uma molécula externa (ligante) e inclusão dos parâmetros para fazer a simulação  (montagem do complexo).
* Relembrar as etapas da simulação e execução dos cálculos.

No dia 03 discutiremos
* Análises das simulações de DM
    * Visualizar as trajetórias.
    * Análisas "energias".
    * Mapeas as propriedades estruturais.

No dia 04 discutiremos
* Análises das simulações de DM
    * Estimativa de energia livre de ligação pelo método MMGBSA.


### Parte 0 - Pasta de trabalho:
```
/home/ixemmsb/EMMSB_2018_DM/02-proteina/
```

Aqui você vai encontrar o arquivo de estrutura da HIV-1 protease em complexo com o inibidor <b>Viracept</b> baixado direto do Protein Databank neste link
https://www.rcsb.org/structure/1OHR

| arquivo | descrição | 
| --- | --- |
|1ohr.pdb | Arquivo .PDB original da HIV-1 Protease |
|1ohr_proa.pdb |Cadeia A da proteína|
|1ohr_prob.pdb |Cadeia B da proteína|
|1ohr_heta.pdb | Hetero átomos |
|1ohr.vmd | Script do VMD para visualisar a proteina |

Explore a estrutura usando o <b>VMD</b>.  
Para agilizar a visualização você pode usar o comando:
```
vmd -e 1ohr.vmd
```

## Parte 1 até 8, pastas de simulação.
Para facilitar a sua vida, organizamos as diferentes etapas das preparação e simulação em pastas numeradas.

|Passo| Nome da pasta |
|---|---|
|1|01.preparacao_da_proteina/ |
|2|02.preparacao_do_ligante/  |
|3|03.montagem_do_complexo/   |
|4|04.preparacao_da_simulacao/ |
|5|05.minimizacao/              |
|6|06.equilibracao/            |
|7.1|07.producao/                |
|7.2|07.producao.npt/           |
|8|08.analises/  |

### Parte 1 - "01.preparacao_da_proteina/" 
* O arquivos de difração da raios-X não contém os átomos de Hidrogênio.
* Como a protonação depende do microambiente químico ao redor do grupo titulável, precisamos de um método para prever corretamente dos estados de protonação dos resíduos de aminoáciodo.
O resíduo de Histidina é terror dos modeladores, lembre-se como é a curva de titulação. (copy&paste do Lehninger)


![title](imagens/titulacao_histidina.png)



* Use o programa PROPKa para fazer a previsão dos estados de protonação.

http://nbcr-222.ucsd.edu/pdb2pqr_2.0.0/

#### Se tudo der certo você terá como output uma tela assim:
![title](01.preparacao_da_proteina/propka_tela.png)


#### Baixe os arquivos e vamos ver o resultado
O arquivo .pqr terá as novas coordenadas dos átomos de hidrogênio para os resíduos que o PROPKa reconhecer. Ele irá te avisar caso existam resíduos sem referência no banco de dados. Se for o caso, estes serão ignorados... fique atento !
Abaixo um trecho do arquivo <b>"1ohr.pqr"</b>

![title](01.preparacao_da_proteina/propka_pqr.png)



O arquivo .propka possui "log" contendo os detalhes da previsão. Lembre-se de ler atentamente o log e investigar os resíduos. 
Abaixo um trecho do arquivo <b>"1ohr.propka"</b>

![title](01.preparacao_da_proteina/propka_log.png)


Para facilitar a sua vida, após a descrição detalhada das interações o .propra apresenta um resumo dos pKas preditos e dos pKas de referência, comece a investigar a proteína pelos resíduos com a maior diferença de pKa.

Repare o ASP 25 (cadeia A), que se repete mais abaixo (a cadeia B). Nesta HIV-1 protease, apenas um deles deve estar protonado.  
Abaixo mais um trecho do arquivo <b>"1ohr.propka"</b>

![title](01.preparacao_da_proteina/propka_summary.png)



#### Idealmente você deve investigar todos desvios de pKa que afetem o estado de protonação das cadeias laterais dos resíduos da sua proteínas <i> no pH que você escolheu simular</i>
Visualize pelo menos os dois ASP 25 (no centro da cavidade) para compreender os resultados do PROPKa.
```
vmd -e proteina.vmd 
```



## Preparação das COORDENADAS e TOPOLOGIA proteina no formato GROMACS.
É muito comum encontrar erros bobos ao trabalhar com os arquivos de entrada .PDB, pois até não muito tempo atrás não existia um PADRÃO bem definido para os nomes de átomos e resíduos. Ainda hoje existem muitos programas, como o PROPKA, que não foram atualizados.

Execute o script 2x, cada vez selecionando uma das duas opções
```bash
bash 01.comando.vai.dar.erro.bash 
```
Os problemas foram corrigidos no arquivo "proteina.pdb", agora use o script que funciona.
```bash
# Roda o pdb2gmx para preparar a proteína para o campo de forças "AMBER99SB-ILDN protein"
bash 02.agora.vai.bash 
```

Os arquivos de saída são:

|Arquivo|Descrição|
|---|---|
|proteina.gro|Coordenadas no formato GROMACS|
|proteina.top|Topologia, como a proteína possui duas cadeia, este arquivo "inclui" dois arquivos separados com a topologia de cada cadeia  |
|proteina_Protein.itp| Topologia da cadeia A |
|proteina_Protein2.itp| Topologia da cadeia B |

Dois novos arquivos também são criados, eles podem ser usados para impor restrições de posição.

|Arquivo|Descrição|
|---|---|
|posre_Protein.itp| Lista da átomos e potencial de restrição  |
|posre_Protein2.itp| Lista da átomos e potencial de restrição |

![title](imagens/posre.png)

As restrições de posição podem ser ativadas no arquivo com os parâmetros de simulação <b>".mdp"</b> usando a flag <b>"-DPOSRES"</b>, sempre que você usar o "gmx grompp". A condição "if" é ativada lá nas últimas linhas dos arquivos proteina_Protein.itp e proteina_Protein2.itp.
![title](imagens/ifposre.png)





### Parte 2 - "02.preparacao_do_ligante" 
As proteínas são geralmente compostas pelos mesmos 20 aminoácidos e umas poucas modificações pós translacionais, a mesma situação se aplica para os ácidos nucleicos e muitos lipídios. As topologias e parâmetros para esses "fragmentos" estão consolidados nos principais campos de forças para simulações biomoleculares.

Os Heteroátomos, no entanto podem ser "qualquer coisa", portanto é impráticável manter bancos de parâmetros e topologias para todas moléculas possíveis, muito mais por que não existem dados experimentais para validadar os parâmetros. (A parametrização CORRETA é extremamente difícil).

Para aliviar esse problema, seguindo a filosofia da transferabilidade de parâmetros entre grupos químicos similares foram desenvolvidos campos de força genéricos (general forcefield), na área biomolecular se destacam o "General Amber Force Field" (GAFF), "CHARMM General Force Field" (CGenFF) e MKTOP.

Os campos de forças genéricos possuem um número maior de TIPOS de átomos, especificamente parametrizados a partir de pequenas moléculas, no caso GAFF, CGenFF e MKTOP, o conjunto de treinamento foi reforçado com moléculas "drug-like".

Baseado na vizinhança (quem está ligado com quem, e a ordem da ligação) os programas escolhem autoMÁGICAMENTE o TIPO de átomo mais adequado. Na prática, se essa seleção ocorrer corretamente, a etapa limitante é o cálculo das cargas parciais atômicas que, segundo a filosofia de parametrização dos campos de forças, deve ser por cálculos quanticos BEEEEM pesados.

Nesta pasta você encontrará EXEMPLOS de arquivos de saída do programa "acpype", desenvolvido originalmente pelo Dr. Alan Wilter. O acpype facilita o uso das ferramentas do programa "AMBER" para criar os parâmetros e escreve os arquivos de saída (coordenadas e topologia) para os mais populares programas de dinamica molecular.

Sousa da Silva, A. W., & Vranken, W. F. (2012). ACPYPE - AnteChamber PYthon Parser interfacE.  
https://doi.org/10.1186/1756-0500-5-367

http://webapps.ccpn.ac.uk/acpype/

### Se o LNCC tivesse instalado, você usaria assim:
```acpype.py -i 1UN.mol2 ```

Ele escreve os arquivos de saída na pasta: ```1UN.acpype/```

Os arquivos que vamos usar são:
```1UN_GMX.gro``` e ```1UN_GMX.itp```

### (Curiosidade) Diferenças nos formatos dos arquivos de topologia e parâmetros
Como ilustração, veja como a diferença no formato dos programas GROMACS e CHARMM.

|Arquivo| Descrição |
|---|---|
|ligante.top | Topologia  no formato GROMACS|
|ligante.itp | Parâmetros no formato GROMACS|
|ligante.rtf | Topologia  no formato CHARMM|
|ligante.prm | Parâmetros no formato CHARMM|




### Parte 3 - "03.montagem_do_complexo" 
Os arquivos de coordenadas, parâmetros e topologia para o <b>Viracept</b> para esta prática estão na pasta ```1UN.acpype/``` mas não se preocupe, eles já foram copiados para "03.montagem_do_complexo".


```bash
#1 criar uma copia da proteina
cp proteina.gro complexo.gro

#2 copiar APENAS OS ATOMOS do arquivo "1UN_GMX.gro" para o final do "complexo.gro"
#  Coloque os atomos ENTRE A PROTEINA E O SOLVENTE (agua) 
gedit 1UN_GMX.gro complexo.gro 


#3 criar uma copia do arquivo de topologia 
cp proteina.top complexo.top

#4 Incluir a topologia do ligante no complexo.top
# Precisa colocar LOGO ABAIXO do campo de forcas pois sao novos parametros.
# Deve ficar como as 3 linhas abaixo

; Include forcefield parameters
#include "amber99sb-ildn.ff/forcefield.itp"
##include "1UN_GMX.itp"

#5 Teste se funcionou 
gmx grompp -v -f addion.mdp -c complexo.gro -p complexo.top
```

### Parte 4 - 04.preparacao_da_simulacao

```bash

# Centraliza (-c) o complexo (.gro) e expande as dimensões do sistema 1.0nm além do complexo.
gmx editconf -c -f complexo.gro -o caixa.gro -d 1.0 

# Copie a complexo.top para sistema.top   ( apenas para preservarmos os arquivos)
cp complexo.top caixa.top

# Solvata o complexo com moléculas de água.
gmx solvate -cs -cp caixa.gro -p caixa.top -o caixa.gro

# Use o script abaixo para ver a proteina muito linda.
vmd -e caixa.vmd 



# Preparar para adicionar os ions
gmx grompp -v -f addion.mdp -c caixa.gro -p caixa.top -o addion.tpr

# Copie a caixa.top para sistema.top   ( apenas para preservarmos os arquivos)
cp caixa.top sistema.top

# Neutralizar o sistema substituindo agua por ions, e deixar com concentracao de 150 mM de NaCl
gmx genion -s addion.tpr -p sistema.top -o sistema.gro -neutral -conc 0.15 -pname NA -nname CL

# No metodo lindo:
vmd -e sistema.vmd
```

### Parte 5 - "05.minimizacao"
```bash
# Veja o arquivo de minimizacao com restricoes de posicoes para a proteina
gedit mini_pr.mdp

# Prepare a minimização com restrições de posição "mini_pr".
gmx grompp -v -f mini_pr.mdp -c sistema.gro -p sistema.top -o mini_pr.tpr 
gmx mdrun -v -deffnm mini_pr

# Analise a energia e plote o gráfico da energia potencial.
gmx energy -f mini_pr.edr -w 

# Veja o arquivo de minimizacao SEM restrições de posição.
gedit mini.mdp

# Prepare a Minimizacao.
gmx grompp -v -f mini.mdp -c mini_pr.gro -t mini_pr.trr -p sistema.top -o mini.tpr
gmx mdrun -v -deffnm mini

# Resultado para a energia potencial. A energia convergiu agora ?
echo Potential | gmx energy -f mini.edr -w 

```

### Parte 6 - "06.equilibracao"
```bash
# prepara a equilibracao
gedit equil.mdp

# Repare que no controle de temperatura tem dois grupos, Solute e Solvente 
# Eu criei um arquivo de indice (index.ndx) contendo esses grupos.

gmx make_ndx -f ../05.minimizacao/mini.gro  -o index.ndx 

# 1) escolhi juntar "Proteina" com "Ligante"
# 2) escolhi juntar "Agua" com Ions
# 3) renomeei novos grupos
# Veja abaixo

> 1|13

Copied index group 1 'Protein'
Copied index group 13 'LIG'
Merged two groups with OR: 3128 85 -> 3213

 24 Protein_LIG         :  3213 atoms

> 17|19

Copied index group 17 'SOL'
Copied index group 19 'Ion'
Merged two groups with OR: 24294 54 -> 24348

 25 SOL_Ion             : 24348 atoms

> name 24 solute

> name 25 solvent

> quit



# Agora prepare o arquivo da equilibracao
gmx grompp -v -f equil.mdp -c ../05.minimizacao/mini.gro -t ../05.minimizacao/mini.trr -p ../04.preparacao_da_simulacao/sistema.top -o equil.tpr -n index.ndx

# Agora e so executar a simulacao  (Ja esta pronto nao aperte enter !!!! )
gmx mdrun -v -deffnm equil


# Hora de visualizar
vmd 

# Equilibracao da pressao
gmx grompp -v -f equil.npt.mdp -c equil.gro -t equil.cpt  -p ../04.preparacao_da_simulacao/sistema.top  -o equil.npt.tpr -n index.ndx
gmx mdrun -v -deffnm equil.npt
```


### Parte 7 - "07.produção" 
Já sabe o que fazer !

```bash
gmx grompp -v -f prod.mdp -c ../06.equilibracao/equil.gro -t ../06.equilibracao/equil.cpt  -p ../04.preparacao_da_simulacao/sistema.top -o prod.tpr -n ../06.equilibracao/index.ndx 
gmx mdrun -v -deffnm prod
```

### Parte 7 - "07.produção.npt" 
```bash
gmx grompp -v -f prod.mdp -c ../06.equilibracao/equil.npt.gro -t ../06.equilibracao/equil.npt.cpt  -p ../04.preparacao_da_simulacao/sistema.top -o prod.tpr -n ../06.equilibracao/index.ndx 
gmx mdrun -v -deffnm prod

```

# Parte 8 - Vá para a pasta "08.analises"
## Análises das trajetórias de Dinâmica Molecular
Até agora já vimos as análises de temperatura, pressão e densidade para as caixas d'água e vocês notam que a mesmo num sistema simples como a água a convergência das medidas pode ser demorada, além disso o valor médio da propriedade pode ser diferente do valor de referência, pois os campo de forças são parametrizados para reproduzir um número limitado de propriedades.

O sistema proteína-ligante é bem mais complexo, isso dificulta a convergências das propriedades, portanto antes de realmente considerar a sua simulação como <b>produção</b> assegure-se que as suas métricas de interesse convergiram.

### A seguir, vamos analisar as propriedades estruturais da proteína, ligante e complexo a partir das trajetórias.
* Propriedades da proteína
    * RMSD em relação a uma estrutura de referência (inicial).
    * RMSF em relação a estrutura de referência (média).
    * Conteúdo de estrutura secundária.
    * Número de ligações Hidrogênio
* Propriedades do ligante
    * RMSD do ligante
    * RMSF do ligante
* Propriedades do complexo proteína-ligante
    * Número de ligações Hidrogênio
    * Pontes salinas

(Quinta-feira : Finalmente estimaremos a <b>Energia livre de ligação</b> com o método MM/GBSA )



### Visualize a trajetória. Aproveite para representá-la de uma forma informativa (bonita)
vmd -e abrebembonito.vmd


### Sigam o Reinaldo agora, vamos calcular algumas coisas usando o VMD
* A sobreposição (fit) da trajetória, e o RMSD.
* Número ligações Hidrogênio

Agora é com vocês:
* Número ligações Hidrogênio Proteína-Ligante
* Número de Pontes Salinas
    * proteína-proteína
    * proteína-ligante
* ... seja criativo com as opções disponíveis.


# Agora voltando para a linha de comando... :(

### Calcule o RMSD para o backbone
Escolha "Backbone" para fazer o "fit", em seguida escolha "Protein" como grupo de cálculo.
```bash
#RMSD
gmx rms -s prod.tpr  -f prod.xtc -o rmsd.xvg 

#Visualize com o programa XMGRACE:
xmgrace rmsd.xvg
```

### Calcule o RMSF para os <b>resíduos</b> de aminoácido da proteína.
Escolha "Protein-H", assim você exclui os átomos de Hidrogênio.
Com a opção <b>-oq</b> nós criamos um arquivo .PDB (bfactor.pdb) com os valores do RMSF escritos no campo do "b-factor"
```bash
gmx rmsf -s ../07.producao.npt/prod.tpr  -f ../07.producao.npt/prod.xtc -oq bfactor -fit -res

xmgrace rmsf.xvg 
```
### Use o programa <b>pymol</b> para visualizar os regiões mais flexíveis.
No menu ao lado do objeto "bfactor", clique no botão "<b>A</b>" - > "<b>presets</b>" -> "<b>b-factor putty</b>". 
```bash
pymol bfactor.pdb 
```

![title](imagens/pymol_makebfactor.png)


### Calcula a estrutura secundária da proteína.
Apos digitar o comando abaixo, escolha "Protein"

```bash
# Computa a estrutura secundária
gmx do_dssp -s prod.tpr -f prod.xtc -tu ns 

# Visualize o conteúdo de estrutura secundária
xmgrace -nxy scount.xvg

# Converte o arquivo .xpm em .eps (figura)
gmx xpm2ps -f ss.xpm -o ss.eps

# Visualize com o programa "eog"
eog ss.eps
```





# Estimativa de energia livre de ligação com o método MM/PBSA
Agora que temos a simulação convergida (#sqn) podemos podemos estimar propriedades interessantes para o descobrimento de drogas/estabilidade. 

Vamos discutir a variação de energia livre de Gibbs por conta da formação do complexo proteína-ligante.

Vamos ver alguns slides antes...
![title](imagens/APAGAR_MMPBSA_EMMSB_2018.png)



## Neste processo, vamos sofrer mais um pouco... (#risadasinistra)
A ferramenta para estimar esta propriedade não possui uma implementação nativa no GROMACS. 
Vamos usar uma implementada no pr
ograma da concorrência, o <b>Amber</b>.

Faremos os seguintes passos:
* Converter a topologia e as coordenadas do COMPLEXO do formato GROMACS para AMBER
* Converter a trajetória da simulação (prod.xtc) para uma "fitted" (prod_fit.xtc).
* Converter a trajetória de simulação para o formato AMBER, além disso tirar o solvente.

Para o Cálculo MMPBSA precisamos também:
* Produzir topologias independentes (AMBER) para o complexo (com.top) receptor (rec.top) e ligante (lig.top)
* Correr o cálculo.


```bash
# Usar o pacote python <b>PARMED</b> para converter as coordenadas e topologia do complexo.
amber.python converte_topologia_do_complexo.py

# Converter a trajetória da simulação (prod.xtc) para uma "fitted" (prod_fit.xtc)
# Escolha o Backbone para o "fit" e o "System" para o arquivo de saída.
gmx trjconv -s prod.tpr -f prod.xtc -o prod_fit.xtc -fit rot+trans

# Converter a trajetória de simulação para o formato AMBER, além disso tirar o solvente.
cpptraj -i cpptraj.in
```

In [1]:
### Agora  leia o arquivo "leiame.txt"
